In [6]:
import warnings
warnings.filterwarnings('ignore')

# 회원 탈퇴 예측
* 탈퇴한 회원과 계속 이용하는 회원의 데이터를 가지고 '의사결정 트리(Decision Tree)' 알고리즘을 사용해서 지도학습의 분류 알고리즘을 이용해서 탈퇴를 예측

### 1 : 데이터 읽고 이용데이터 수정

In [1]:
# use_log_months.csv : 작성한 이용 이력을 연월/고객별로 집계한 데이터
# customer_join.csv : 이용 이력을 포함한 고객 데이터

import pandas as pd

customer = pd.read_csv('./Data-in/customer_join.csv')
uselog_months = pd.read_csv('./Data-in/use_log_months.csv')

In [2]:
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,야간,6000,일반,3.916667,4.0,6,1,1,2019-04-30,47


In [3]:
uselog_months.head()

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


* 데이터 가공

In [7]:
# 미래를 예측하기 위해 그달과 1개월 전의 이용 이력으로 데이터 작성
year_months = list(uselog_months["연월"].unique())
uselog = pd.DataFrame()

# 2018/5월부터 for문을 반복
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["연월"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    # 2018/4월 데이터(tmp가 5월 데이터일 경우)
    tmp_before = uselog_months.loc[uselog_months["연월"]==year_months[i-1]]
    del tmp_before["연월"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()
    

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### 2 : 탈퇴전월의 탈퇴고객데이터 작성

In [9]:
# 탈퇴월 : end_date 컬럼 참고, 그러나 탈퇴를 예측하는 목적은 탈퇴를 미연에 방지
# 스포츠 센터의 탈퇴 업무 로직은 월말까지 탈퇴 신청을 하면 다음달에 탈퇴 처리
# 9월 end_date 탈퇴 정보가 있다고 하면, 8월에 탈퇴 신청을 한거기 때문에 
    # 분석을 9월에 한다면 탈퇴를 미연에 방지하지 못함
    # 탈퇴 월을 2018/8월로 세팅해서 처리, 실제 분석 월은 7월 데이터로 8월의 탈퇴 신청 확률 예측   
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None

In [11]:
# end_date의 1개월 전을 계산해서 연월에 저장한 후 uselog와 customer_id, 연월을 키로 결합
# to_datetime : 날짜를 계산하기 위해 사용
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] \
        - relativedelta(months=1)  # 1개월 전의 날짜를 저장
exit_customer["연월"] = pd.to_datetime(exit_customer["exit_date"]).dt.\
    strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str)

exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "연월"], \
    how="left")
print(len(uselog))
exit_uselog.head()

33851


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 결합한 데이터가 탈퇴한 회원의 탈퇴 전월의 데이터 뿐이기 때문에 결측치가 많음
# 결측치가 없는 데이터만 남김

# 어떤 특정 회원이 그만두기 전월의 상태를 나타내는 데이터

# ** 탈퇴 회원의 데이터를 작성함 **
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

### 3 : 지속회원의 데이터 작성

### 4 : 예측할 달의 재적기간 작성

### 5 : 결측치 제거

### 6 : 문자열 변수를 처리할수 있도록 가공

### 7 : 의사결정나무를 사용해서 탈퇴예측모델을 작성하자